In [1]:
import torch
import torchvision
import torch.nn as nn
import torch.nn.functional as F
from torchvision import datasets 
from torchvision import transforms
from torchvision.utils import save_image
from torch.autograd import Variable

In [2]:
def to_var(x):
    if torch.cuda.is_available():
        x = x.cuda()
    return Variable(x)

In [3]:
def denorm(x):
    out = (x + 1) / 2
    return out.clamp(0, 1)

In [4]:
# Image processing
transform = transforms.Compose([
    transforms.ToTensor(),
    transforms.Normalize(mean=(0.5, 0.5, 0.5), std=(0.5, 0.5, 0.5))
])

In [5]:
# MNIST dataset
mnist = datasets.MNIST(
    root='../../data/', train=True, transform=transform, download=True)
# Data loader
data_loader = torch.utils.data.DataLoader(
    dataset=mnist, batch_size=100, shuffle=True)

In [9]:
# Discriminator
D = nn.Sequential(
    nn.Linear(784, 256), nn.LeakyReLU(0.2), nn.Linear(256, 256),
    nn.LeakyReLU(0.2), nn.Linear(256, 1), nn.Sigmoid())

# Generator
G = nn.Sequential(
    nn.Linear(64, 256), nn.LeakyReLU(0.2), nn.Linear(256, 256),
    nn.LeakyReLU(0.2), nn.Linear(256, 784), nn.Tanh())

In [10]:
if torch.cuda.is_available():
    D.cuda()
    G.cuda()

In [11]:
# Binary cross entropy loss and optimizer
criterion = nn.BCELoss()
d_optimizer = torch.optim.Adam(D.parameters(), lr=0.0003)
g_optimizer = torch.optim.Adam(G.parameters(), lr=0.0003)

In [ ]:
# Start training
for epoch in range(200):
    for i, (images, _) in enumerate(data_loader):
        # Build mini-batch dataset
        batch_size = images.size(0)
        images = to_var(images.view(batch_size, -1))

        # Create the labels which are later used as input for the BCE loss
        real_labels = to_var(torch.ones(batch_size))
        fake_labels = to_var(torch.zeros(batch_size))

        #============= Train the discriminator =============#
        # Compute BCE_Loss using real images where BCE_Loss(x, y): - y * log(D(x)) - (1-y) * log(1 - D(x))
        # Second term of the loss is always zero since real_labels == 1
        outputs = D(images)
        d_loss_real = criterion(outputs, real_labels)
        real_score = outputs

        # Compute BCELoss using fake images
        # First term of the loss is always zero since fake_labels == 0
        z = to_var(torch.randn(batch_size, 64))
        fake_images = G(z)
        outputs = D(fake_images)
        d_loss_fake = criterion(outputs, fake_labels)
        fake_score = outputs

        # Backprop + Optimize
        d_loss = d_loss_real + d_loss_fake
        D.zero_grad()
        d_loss.backward()
        d_optimizer.step()

        #=============== Train the generator ===============#
        # Compute loss with fake images
        z = to_var(torch.randn(batch_size, 64))
        fake_images = G(z)
        outputs = D(fake_images)

        # We train G to maximize log(D(G(z)) instead of minimizing log(1-D(G(z)))
        # For the reason, see the last paragraph of section 3. https://arxiv.org/pdf/1406.2661.pdf
        g_loss = criterion(outputs, real_labels)

        # Backprop + Optimize
        D.zero_grad()
        G.zero_grad()
        g_loss.backward()
        g_optimizer.step()

        if (i + 1) % 300 == 0:
            print('Epoch [%d/%d], Step[%d/%d], d_loss: %.4f, '
                  'g_loss: %.4f, D(x): %.2f, D(G(z)): %.2f' %
                  (epoch, 200, i + 1, 600, d_loss.item(), g_loss.item(),
                   real_score.data.mean(), fake_score.data.mean()))

    # Save real images
    if (epoch + 1) == 1:
        images = images.view(images.size(0), 1, 28, 28)
        save_image(denorm(images.data), './data/real_images.png')

    # Save sampled images
    fake_images = fake_images.view(fake_images.size(0), 1, 28, 28)
    save_image(
        denorm(fake_images.data), './data/fake_images-%d.png' % (epoch + 1))

/home/wanglijun/anaconda3/lib/python3.6/site-packages/torch/nn/functional.py:1474: UserWarning: Using a target size (torch.Size([100])) that is different to the input size (torch.Size([100, 1])) is deprecated. Please ensure they have the same size.
  "Please ensure they have the same size.".format(target.size(), input.size()))


Epoch [0/200], Step[300/600], d_loss: 0.5309, g_loss: 2.6516, D(x): 0.79, D(G(z)): 0.19
Epoch [0/200], Step[600/600], d_loss: 2.2574, g_loss: 0.8113, D(x): 0.43, D(G(z)): 0.45
Epoch [1/200], Step[300/600], d_loss: 1.6688, g_loss: 1.4659, D(x): 0.65, D(G(z)): 0.46
Epoch [1/200], Step[600/600], d_loss: 0.1316, g_loss: 4.3763, D(x): 0.96, D(G(z)): 0.07
Epoch [2/200], Step[300/600], d_loss: 0.6292, g_loss: 2.2588, D(x): 0.75, D(G(z)): 0.17
Epoch [2/200], Step[600/600], d_loss: 0.7988, g_loss: 1.7797, D(x): 0.76, D(G(z)): 0.32
Epoch [3/200], Step[300/600], d_loss: 0.7584, g_loss: 2.7641, D(x): 0.80, D(G(z)): 0.29
Epoch [3/200], Step[600/600], d_loss: 1.4283, g_loss: 1.3157, D(x): 0.64, D(G(z)): 0.44
Epoch [4/200], Step[300/600], d_loss: 0.3495, g_loss: 3.2987, D(x): 0.90, D(G(z)): 0.12
Epoch [4/200], Step[600/600], d_loss: 1.2803, g_loss: 1.1740, D(x): 0.59, D(G(z)): 0.40
Epoch [5/200], Step[300/600], d_loss: 1.1722, g_loss: 1.9675, D(x): 0.57, D(G(z)): 0.22
Epoch [5/200], Step[600/600], d_

Save the trained parameters

In [ ]:
torch.save(G.state_dict(), './generator.pkl')
torch.save(D.state_dict(), './discriminator.pkl')